## Let’s recap

In [the fist lab](1-WKSHP-Terraform101-CreatingResources.ipynb) , we covered HPE GreenLake with its Private Cloud Service and showed how to get started with Terraform and the Terraform provider for HPE GreenLake. In this lab, we will start with the same VM configuration created in Lab 1 as shown below. From there we will show you how to tap into more advanced functionality that’s provided by Terraform and the HPE GreenLake provider:

![GreenLake VM created using Terraform](./Pictures/vmcreated.png "GreenLake VM created using Terraform")

First, let's reload our environment variables:

In [2]:
# Source RC file to reload environment variables
source ./student.rc
source ./didier.rc

export HPEGL_TENANT_ID={{ GREENLAKE_TENANT_ID }}
export HPEGL_USER_ID= {{ STUDENT_CLIENTID }}
export HPEGL_USER_SECRET= {{ STUDENT_CLIENTSECRET }}
export HPEGL_IAM_SERVICE_URL={{ GREENLAKE_IAM_API_URL  }}/oauth2/{{ GREENLAKE_TENANT_ID  }}/{{ GREENLAKE_IAM_ID }}
export TF_CLI_ARGS_plan="-parallelism=1"
export TF_CLI_ARGS_apply="-parallelism=1"
#export TF_LOG=DEBUG
export TF_LOG=bash: export: `}}': not a valid identifier
bash: export: `{{': not a valid identifier
bash: export: `}}': not a valid identifier
bash: export: `{{': not a valid identifier
bash: export: `}}': not a valid identifier
bash: export: `}}/oauth2/{{': not a valid identifier
bash: export: `}}/{{': not a valid identifier
bash: export: `}}': not a valid identifier


## Infrastructure lifecycle management

Once your infrastructure is created, it will need to evolve over time in order to cope with any change of workload or to perform maintenance. Let’s look at a few scenarios.

### Use case 1: Setting up tags and labels

As organizations scale their cloud environments, they often need to define methodologies for organizing resources. For this, they can leverage tags and labels. Tags consist of key/value pairs that make it easier to search for, or filter, your cloud resources based on categories relevant to the organization. Another option is to attach labels, which are simple values, to your VMs in order to keep track of what it’s used for or who it belongs to.

So let's try adding metadata to the VM using tags and labels. According to the [documentation](https://github.com/HPE/terraform-provider-hpegl/blob/main/docs/resources/vmaas_instance.md), you can add labels by adding the following snippets to our configuration file:

```markdown
# Using labels
labels = ["hackshack", "hpedev"]
```

And you can add tags by inserting the following other code snippet in your configuration file:

```markdown
# Using tags
tags = {
        team  = "HPE Developer"
        support = "gold"
  }
```

Save file and apply those changes with **terraform apply**, wait a little and look at the VM details. You can see the labels and the tags in the capture below:

![tags and labels applied to VM](/img/terraform-greenlake-part2-blog-picture3.png "tags and labels applied to VM")

### Use case 1: Stop this VM

To start, let’s keep things simple. You might want to just turn off the VMs that are part of an infrastructure when you don’t need them to save cost or limit your carbon impact on the planet. If you paid attention to the current configuration file, you’ll see that we inserted a power statement when originally creating the VM. While *poweron* is the only valid option when creating a new resource of type **hpegl\_vmaas\_instance**, other values are available for lifecycle management, such as *poweroff* and *suspend*.

Locate the following section in your configuration file:

```markdown
     }
     power = "poweron"
   }
```

And change it so that the power desired state is set to *poweroff* as shown below:

```markdown
     }
     power = "poweroff"
   }
```

Save and run a **terraform apply** command:

```markdown
Terraform will perform the following actions:

# hpegl_vmaas_instance.DidierTest1 will be updated in-place
  ~ resource "hpegl_vmaas_instance" "DidierTest1" {
        id                 = "149"
        name               = "DidierTest1"
      ~ power              = "poweron" -> "poweroff"
```

Pretty soon afterwards, you can check out the HPE GreenLake console and see that the VM status was changed to stopped.

![VM is now stopped](/img/terraform-greenlake-part2-blog-picture2-2.png "VM is now stopped")

In [ ]:
# Apply power state changes

### Use case 3: Get me more disk space

Another typical use case would be to add another disk to a VM, say a data volume for application usage. The syntax for this is the same as you used to create the VM with its *root_vol*, already visible in the Storage details of the VM:

![VM was created with one disk: root_vol](/img/terraform-greenlake-part2-blog-picture4.png "VM was created with one disk: root_vol")

Go ahead and add the following code snippet to your configuration file:

```markdown
  # Add another volume
  
  volume {
         name         = "data_vol"
         size         = 25
         datastore_id = "auto"
     }
```

Save the file, apply those changes, wait a little and check the VM storage configuration again:

![VM with two disks](/img/terraform-greenlake-part2-blog-picture5.png "VM with two disks")

### Use case 4: Please snap this VM

Here’s one last use case you can try, which consists of snapshotting the VM. You can do tis by adding the following Terraform code snippet in your configuration file:

```markdown
   snapshot {
    name        = "Snap1
    description = "Snap this VM so we can restart from this state"
  }
```

Save the file, apply those changes, wait a little and check the details of the VM  once again, in the Backups section:

![Snap of VM ready](/img/terraform-greenlake-part2-blog-picture6.png "Snap of VM ready")

## Debugging when things go wrong

In this post, I’ve showed you how to make sure the Terraform configuration file is valid before applying changes using the **terraform validate** command. To see more details during an apply command, you can also enable Terraform debug by simply setting up the TF_LOG environment variable. I suggest setting it up to DEBUG but other supported values are: TRACE, INFO, WARN and ERROR.
 
```markdown
export TF_LOG=DEBUG
```